## Learning Julia 1.0 - Implement Quad-Trees in Julia
Work towards an efficient, geo-spatially aware version. Adapted from GeeksForGeeks datatype deesctiption [here](https://www.geeksforgeeks.org/quad-tree/) and [Wikipedia](https://en.wikipedia.org/wiki/Quadtree#Pseudocode)

In [ ]:
using BenchmarkTools

In [1]:
# Struct && Type Definitions
QT_NODE_CAPACITY = 1;

"Catchall AbstractType for 2D shapes w. 4 corners"
abstract type AbstractBox end

"Points to Query"
struct Coord
    lng :: Float64
    lat :: Float64
end

"Box; used for search on qtBox, shape diagonal defined by 
`SW`, the Southwest-corner, and (SW.x + sideLength, SW.y + sideLength)"
struct Box <: AbstractBox
    SW :: Coord
    sideLength :: Float64
end 


"Same as Box; Adds references to set of points and set of children qtBoxes"
mutable struct qtBox <: AbstractBox
    # TODO: Adapt coords to map onto surface of sphere coordinates
    SW :: Coord
    sideLength :: Float64
    children :: Array{qtBox} # No Pointers :( ?
    points:: Array{Coord}
end 

In [2]:
"Check if Point in a 2D Space"
function checkPoint(p::Coord, region::AbstractBox)::Bool
    return (
        (p.lng > region.SW.lng) & (p.lng < region.SW.lng + region.sideLength) &&
        (p.lat > region.SW.lat) & (p.lat < region.SW.lat + region.sideLength)
    )
    end

"Check if `AbstractBox` shapes Intersect"
function regionOverlap(s0::AbstractBox, s1::AbstractBox)::Bool
    # If one rectangle is on left side of other 
    # OR If one rectangle is above other
    if (
        (s0.SW.lng > (s1.SW.lng + s1.sideLength)) ||
        (s1.SW.lng > (s0.SW.lng + s0.sideLength)) || 
        (s0.SW.lat > (s1.SW.lat + s1.sideLength)) || 
        (s1.SW.lat > (s0.SW.lat + s0.sideLength))
        )
        return false
    else 
        return true
    end
end


"Inplace split qtBox into 4 equal parts"
function subdivide!(r::qtBox)
    if (length(r.children) == 0) #if not yet subdivided
        nodeDelta = r.sideLength / 2
        
        # Add each of the children to Array...
        
        append!(r.children, 
            Array([
             # S.W Child (+0, +0)
            qtBox(r.SW, nodeDelta, Array(qtBox[]),  Array(Coord[])),
            
            # S.E Child (+1 , +0)
            qtBox(Coord(r.SW.lng + nodeDelta, r.SW.lat),
                nodeDelta , Array(qtBox[]),  Array(Coord[])),
            
            #N.E child (+1, +1)
            qtBox(Coord(r.SW.lng + nodeDelta, r.SW.lat + nodeDelta),
                nodeDelta , Array(qtBox[]),  Array(Coord[])),
                
            #N.W child (+0, +1)
            qtBox(Coord(r.SW.lng , r.SW.lat + nodeDelta),
                nodeDelta , Array(qtBox[]),  Array(Coord[]))
            ])
        )
        end 
    end 

subdivide! (generic function with 1 method)

In [3]:
#= Random Point Generation/Benchmarking/Testing=#
"""
Generate uniform random points along a surface
e.g. generateRandUniform(0., 10., 0., 10., 1) -> Array{Float64,2}:  2.19137  1.57839
"""
function generateRandUniform(x0::Float64, x1::Float64, y0::Float64, y1::Float64, n::Int)::Array{Coord}
    X = ((ones(n) * x0) + (rand(n) * (x1 - x0))) 
    Y = ((ones(n) * y0) + (rand(n) * (y1 - y0)))
    return [Coord(x, y) for (x,y) in zip(X,Y)]
end

generateRandUniform

In [4]:
"See Wikipedia Description, recursivly query a range on a `qtBox`"
function queryRange(r::qtBox, qryBox::AbstractBox)::Array{Coord}

    # Initialize return list
    pointsInRange = Array{Coord}(Array[])
    
    # Automatically abort if the range does not intersect this quad
    if !regionOverlap(r, qryBox)
        return Array{Coord}(Array[])
    end

    # Check objects overlap at this quad level..
    for p in r.points
        if checkPoint(p, qryBox)
            push!(pointsInRange, p)
        end
    end 
    
    # Terminate here, if there are no children
    if length(r.children) == 0
        return pointsInRange;
    else 
        ## Otherwise, add the points from the children
        for c in r.children
            append!(pointsInRange, queryRange(c, qryBox))
        end
    end
    
    return pointsInRange
end

queryRange (generic function with 1 method)

In [5]:
"See Wikipedia Description, recursivly insert points into range on a `qtBox`"
function insertIntoQuadTree!(r::qtBox, ps::Coord)::Bool        
    # Handle Insert Logic if point is in range of current box
    if !checkPoint(ps, r)
        return false
    end

    # Current box has room && Not subcivided 
    if (length(r.points) < QT_NODE_CAPACITY) && (length(r.children) == 0)
        push!(r.points, ps)
        return true
    end

    if (length(r.children) == 0) #subdivide if is not subdiv'd yet
        subdivide!(r)
    end
    
    for c in r.children  #Check to see if we can insert into any children
        if insertIntoQuadTree!(c, ps)
            return true
        end
    end
    
    return false #Fatal
end

insertIntoQuadTree! (generic function with 1 method)

In [8]:
# Generate Test Set
ps = generateRandUniform(0., 1., 0., 1., 5000)
r = qtBox(Coord(0, 0), 1, Array(qtBox[]), Array(Coord[]))


# Insert
@benchmark (for p in ps insertIntoQuadTree!(r, p) end);

# Sample Query
searchRegion = Box(Coord(rand(), rand()), rand())
@benchmark res = queryRange(r, searchRegion);

BenchmarkTools.Trial: 
  memory estimate:  382.66 KiB
  allocs estimate:  14490
  --------------
  minimum time:     80.727 ms (0.00% GC)
  median time:      84.970 ms (0.00% GC)
  mean time:        84.971 ms (0.00% GC)
  maximum time:     88.393 ms (0.00% GC)
  --------------
  samples:          59
  evals/sample:     1

For posterity's sake...sure these'll come down as this all makes more sense to me...


**Insert 5k obj**
```
BenchmarkTools.Trial: 
  memory estimate:  382.66 KiB
  allocs estimate:  14490
  --------------
  minimum time:     80.727 ms (0.00% GC)
  median time:      84.970 ms (0.00% GC)
  mean time:        84.971 ms (0.00% GC)
  maximum time:     88.393 ms (0.00% GC)
  --------------
  samples:          59
  evals/sample:     1
```
**Query:**
```
BenchmarkTools.Trial: 
  memory estimate:  31.82 MiB
  allocs estimate:  270203 
  --------------
  minimum time:     30.568 ms (0.00% GC)
  median time:      35.957 ms (0.00% GC)
  mean time:        40.934 ms (13.83% GC)
  maximum time:     65.053 ms (32.57% GC)
  --------------
  samples:          123
  evals/sample:     1
```